In [1]:
url = 'http://www.asst-pg23.it/upload/Bandi/avcpLegge190_ASSTPG23_2018_1.xml'

from urllib.request import urlopen

with urlopen(url) as inf:
    data = inf.read()

In [6]:
from lxml import etree

tree = etree.fromstring(data)

In [41]:
lotti = []

for lotto in tree.xpath('//data/lotto'):
    lotto_dict = {
        'importoAggiudicazione': float(lotto.xpath('importoAggiudicazione')[0].text),
        'cig': lotto.xpath('cig')[0].text
    }
    aggiudicatari = []
    for agg in lotto.xpath('aggiudicatari/aggiudicatario'):
        try:
            cf = agg.xpath('codiceFiscale')[0].text
        except IndexError:
            cf = None
        aggiudicatari.append(
            {
                'codiceFiscale': cf,
                'ragioneSociale': agg.xpath('ragioneSociale')[0].text}
        )
    lotto_dict['aggiudicatari'] = aggiudicatari
    lotti.append(lotto_dict)

In [ ]:
for idx, lotto in enumerate(lotti):
    if len(lotto['aggiudicatari']) > 3: 
        print(idx)

In [49]:
lotti[855]        

{'importoAggiudicazione': 39900.0,
 'cig': 'ZD524CCFD9',
 'aggiudicatari': [{'codiceFiscale': '00726860158',
   'ragioneSociale': 'DENTSPLY ITALIA SRL'},
  {'codiceFiscale': '93517310152', 'ragioneSociale': 'HOLLISTER SPA'},
  {'codiceFiscale': '00674840152', 'ragioneSociale': 'B BRAUN MILANO  SPA'},
  {'codiceFiscale': '06324460150', 'ragioneSociale': 'TELEFLEX MEDICAL SRL'}]}

In [ ]:
from IPython.display import HTML

header = '<table class="table"><tr><th>cig<th>importoAggiudicazione<th>codiceFiscale<th>ragioneSociale'

table = []
for lotto in lotti:
    prima_riga = True
    aggiudicatari = lotto['aggiudicatari']
    rows = []
    for aggiudicatario in aggiudicatari:
        if prima_riga: 
            prima_riga = False
            row = f"""
                <td rowspan="{len(aggiudicatari)}">{lotto['cig']}
                <td rowspan="{len(aggiudicatari)}">{lotto['importoAggiudicazione']}
            """
        else:
            row = ''
        row += f"""
            <td>{aggiudicatario['codiceFiscale']}
            <td>{aggiudicatario['ragioneSociale']}
        """
        rows.append(row)
    table.append('<tr>' + '<tr>'.join(rows))

html = header + ''.join(table) + '</table>'
HTML(html)

In [82]:
title = 'Bandi'

with open('bandi.html', 'w') as ouf: ouf.write(f"""
<!doctype html>
<html>
    <head>
        <meta charset="utf-8">
        <title>{title}</title>
            <script src="sorttable.js"></script>
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-beta/css/bootstrap.min.css" integrity="sha384-/Y6pD6FV/Vv2HJnA6t+vslU6fwYXjCFtcEpHbNJ0lyAFsXTsjBbfaDjzALeQsN6M" crossorigin="anonymous">
        <style>
            body {{
                    padding-top: 2rem;
                    padding-bottom: 2rem;
                }}
        </style>
    </head>
    <body>{html}
    </body>
</html>
""")